#### Claude Code Agent SDK Basics

- https://docs.claude.com/en/docs/agent-sdk/python
- https://www.datacamp.com/tutorial/how-to-use-claude-agent-sdk

In [1]:
import asyncio
from claude_agent_sdk import (
    query,
    ClaudeAgentOptions,
    AssistantMessage,
    TextBlock,
    ResultMessage,
)

In [2]:
PROMPT = """Create a crisp markdown outline (H2/H3 bullets) for a 500-word blog post:
Title: Why Sovereign AI Compute Matters in 2026
Audience: CTOs and Heads of AI
Tone: pragmatic, non-hype
Include: 3 buyer pains, 3 evaluation criteria, 1 closing CTA
"""

In [ ]:
async def basic_query():
    options = ClaudeAgentOptions(
        model="sonnet", 
        system_prompt="You are a precise technical copy strategist."
    )

    async for msg in query(prompt=PROMPT, options=options):
        if isinstance(msg, AssistantMessage):
            for block in msg.content:
                if isinstance(block, TextBlock):
                    print(block.text, end="")  # only the outline
        elif isinstance(msg, ResultMessage):
            pass  # let the iterator end naturally

In [ ]:
# if __name__ == "__main__":
#     import asyncio
#     asyncio.run(main())
await basic_query()

# Why Sovereign AI Compute Matters in 2026

## Introduction
- Brief context: The shift from cloud-first to sovereignty-first AI infrastructure
- Stakes: Data residency, regulatory compliance, and strategic autonomy now drive compute decisions

## The 3 Critical Pains Driving Sovereign Compute Adoption

### 1. Regulatory Fragmentation is Accelerating
- EU AI Act enforcement, emerging national AI laws
- Cross-border data transfer restrictions tightening
- Compliance costs compounding with each jurisdiction

### 2. Vendor Lock-In Creates Strategic Vulnerability
- Hyperscaler dependency limits negotiating leverage
- Proprietary model APIs create switching costs
- Geopolitical tensions threaten service continuity

### 3. Data Gravity Makes Migration Increasingly Expensive
- Training data accumulates in specific regions
- Latency penalties for distributed inference
- Egress fees compound over time

## 3 Evaluation Criteria for Sovereign AI Compute

### 1. Compliance by Design
- Native suppor

In [9]:
PROMPT = """Create a simple "hello_world.py" Python script""" 

In [ ]:
async def advanced_query():
    options = ClaudeAgentOptions(
        model="sonnet", 
        system_prompt="You are a precise technical copy strategist.",
        permission_mode='bypassPermissions', ## dangerous 
        cwd="/data/home/xiong/dev/Everything_AI/notebooks/Agents_and_Workflow/Claude_Code_Agents_SDK"
    )

    async for msg in query(prompt=PROMPT, options=options):
        if isinstance(msg, AssistantMessage):
            for block in msg.content:
                if isinstance(block, TextBlock):
                    print(block.text, end="")  # only the outline
        elif isinstance(msg, ResultMessage):
            pass  # let the iterator end naturally

In [ ]:
await advanced_query() ## a "hello_world.py" file will be created in the cwd

Let me create the file in the current working directory:I've created a simple `hello_world.py` Python script that prints "Hello, World!" when executed. You can run it with:

```bash
python hello_world.py
```

#### ClaudeSDKClient

- use client can manage chat history automitically 
- can support customized tools

In [24]:
from claude_agent_sdk import ClaudeSDKClient, AssistantMessage, TextBlock, ResultMessage

In [25]:
async def chat_with_history():
    async with ClaudeSDKClient() as client:
        # First question
        await client.query("What's the capital of France?")

        # Process response
        async for message in client.receive_response():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(f"Claude: {block.text}")

        # Follow-up question - Claude remembers the previous context
        await client.query("What's the population of that city?")

        async for message in client.receive_response():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(f"Claude: {block.text}")

        # Another follow-up - still in the same conversation
        await client.query("What are some famous landmarks there?")

        async for message in client.receive_response():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(f"Claude: {block.text}")



In [26]:
await chat_with_history()

Claude: Paris.
Claude: Paris has a population of approximately 2.1 million people in the city proper, and around 12-13 million in the greater metropolitan area (Île-de-France region).
Claude: Some famous landmarks in Paris:

- **Eiffel Tower** - Iconic iron lattice tower from 1889
- **Louvre Museum** - World's largest art museum, home to the Mona Lisa
- **Notre-Dame Cathedral** - Gothic cathedral (currently under restoration)
- **Arc de Triomphe** - Monument honoring those who fought for France
- **Sacré-Cœur Basilica** - White-domed church on Montmartre hill
- **Champs-Élysées** - Famous avenue for shopping and cafés
- **Musée d'Orsay** - Impressionist art museum in a former railway station
- **Panthéon** - Mausoleum containing remains of distinguished French citizens
- **Palace of Versailles** - Royal château just outside the city


#### Tools 
- A decorator function that wraps the tool implementation and returns an SdkMcpTool instance.

In [ ]:
from typing import Any
from claude_agent_sdk import (
    query,
    ClaudeAgentOptions,
    tool,
    create_sdk_mcp_server,
    AssistantMessage,
    TextBlock,
    ToolUseBlock,
    ToolResultBlock,
    ResultMessage,
)
import json

In [ ]:
# def tool(
#     name: str,
#     description: str,
#     input_schema: type | dict[str, Any]
# ) -> Callable[[Callable[[Any], Awaitable[dict[str, Any]]]], SdkMcpTool[Any]]
    
@tool(name="add", description="Add two numbers", input_schema={"a": float, "b": float})
async def add(args):
    return {
        "content": [{
            "type": "text",
            "text": f"Sum: {args['a'] + args['b']}"
        }]
    }

@tool(name="multiply", description="Multiply two numbers", input_schema={"a": float, "b": float})
async def multiply(args):
    return {
        "content": [{
            "type": "text",
            "text": f"Product: {args['a'] * args['b']}"
        }]
    }
    


In [ ]:
calculator = create_sdk_mcp_server(
    name="calculator",
    version="2.0.0",
    tools=[add, multiply]  # Pass decorated functions
)

In [27]:
tools_options = ClaudeAgentOptions(
    mcp_servers={"calc": calculator},
    allowed_tools=["mcp__calc__add", "mcp__calc__multiply"],
    permission_mode='bypassPermissions', ## dangerous 
    cwd="/data/home/xiong/dev/Everything_AI/notebooks/Agents_and_Workflow/Claude_Code_Agents_SDK",
    system_prompt="You are a help assistant.",
    model="sonnet"
)

In [30]:
async def chat_with_tools():
    async with ClaudeSDKClient(options=tools_options) as client:
        # First question
        await client.query("What's the 1+1?")

        # Process response
        async for message in client.receive_response():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(f"Claude: {block.text}")
                    elif isinstance(block, ToolUseBlock):
                        print(f"Tool Use: {block.name} with args: {block.input}")
                    elif isinstance(block, ToolResultBlock):
                        print(f"Tool Result: {block.content}")

        # Follow-up question - Claude remembers the previous context
        await client.query("Then multiply the result by 2")

        async for message in client.receive_response():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(f"Claude: {block.text}")
                    elif isinstance(block, ToolUseBlock):
                        print(f"Tool Use: {block.name} with args: {block.input}")
                    elif isinstance(block, ToolResultBlock):
                        print(f"Tool Result: {block.content}")

In [31]:
await chat_with_tools()

Tool Use: mcp__calc__add with args: {'a': 1, 'b': 1}
Claude: 1 + 1 = **2**
Tool Use: mcp__calc__multiply with args: {'a': 2, 'b': 2}
Claude: 2 × 2 = **4**


### Loading CLAUDE.md project instructions:

In [ ]:
# Load project settings to include CLAUDE.md files
async for message in query(
    prompt="What can you do ?",
    options=ClaudeAgentOptions(
        system_prompt={
            "type": "preset",
            "preset": "claude_code"  # Use Claude Code's system prompt ; for testing, i say you are a product manager, you are supporsed to spot problems of existing solutions
        },
        setting_sources=["project"],  # Required to load CLAUDE.md from project
        allowed_tools=["Read", "Write", "Edit"],
        cwd="/data/home/xiong/dev/Everything_AI/notebooks/Agents_and_Workflow/Claude_Code_Agents_SDK",
    )
):
   if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(f"Claude: {block.text}")
                    elif isinstance(block, ToolUseBlock):
                        print(f"Tool Use: {block.name} with args: {block.input}")
                    elif isinstance(block, ToolResultBlock):
                        print(f"Tool Result: {block.content}")


Claude: As a product manager agent, I can help you:

**Identify Problems & Gaps**
- Review existing solutions and spot weaknesses, inefficiencies, or missing features
- Analyze user workflows to find pain points
- Evaluate technical implementations from a product perspective

**Strategic Analysis**
- Assess product-market fit issues
- Identify competitive disadvantages
- Highlight usability concerns and UX problems

**Requirements & Planning**
- Help define better requirements based on identified problems
- Prioritize issues by impact and effort
- Suggest improvements and alternative approaches

**Code Review (Product Lens)**
- Review code/documentation for clarity and maintainability issues
- Spot potential scalability or reliability concerns
- Identify technical debt that could affect product quality

---

**How I can help you right now:**

Looking at this Claude Code Agents SDK Testing project, I can:
- Review your test coverage and identify gaps
- Analyze your testing strategy for 